### Helper Function

In [1]:
def check_hand_sign(results):
    """Plot the frame and get the hand sign detected."""
    for result in results:
        frame = result.plot()
        if len(result.boxes.cls) == 1:
            hand_sign = result.names[int(result.boxes.cls)]
        else:
            hand_sign = None
    
    return frame, hand_sign

In [2]:
import cv2
import numpy as np
#ffpyplayer for playing audio
from ffpyplayer.player import MediaPlayer
# video_path="C:\\Users\\benjy\\Downloads\\naruto\\summoning_jutsu_vid.mp4"

def playVideo(video_path):
    video=cv2.VideoCapture(video_path)
    player = MediaPlayer(video_path)
    while True:
        grabbed, frame=video.read()
        audio_frame, val = player.get_frame()
        if not grabbed:
#             print("End of video")
            break
        if cv2.waitKey(28) & 0xFF == ord("q"):
            break
        cv2.imshow("Video", frame)
        if val != 'eof' and audio_frame is not None:
            #audio
            img, t = audio_frame
    video.release()
    cv2.destroyAllWindows()
# PlayVideo(video_path)

### Game Mechanisms

In [3]:
import random

def deploy_damage(hand_sign, player_list, player_id):
    """Deploy Jutsu damage on the opponent."""
    player = player_list[player_id-1]
    other_player_id = 2 if player_id == 1 else 1
    other_player = player_list[other_player_id-1]
    defeated = False
    animate = 0
    if hand_sign is not None:
        for jutsu in player.jutsu_list:
            jutsu.check_pattern(hand_sign, player_id)
            if jutsu.detected:
                print(f'Player {player_id} deployed {jutsu.name}!')
#                 playVideo(jutsu.video_path)
                animate = jutsu.video_path
                
                # 20% damage variation
                damage = int(jutsu.damage*random.uniform(0.8, 1.2))
                other_player.health -= damage
                
                if other_player.health < 0:
                    other_player.health = 0
                    print(f'Player {other_player_id} is defeated!')
                    defeated = True
                else:
                    print(f'Player {other_player_id} suffered {damage} damage and left {other_player.health} hp.\n')
                    
                player.resetJutsu()
                break
    
    return defeated, animate

In [4]:
jutsu_list = []
    
class Jutsu:
    """
    A base class for creating Jutsu.
    
    Attributes:
        name (str): Name of Jutsu technique
        success (list): Jutsu technique pattern.
        damage (int): Amount of damage.
        video_path (Path): Path to the Jutsu animation.
        hand_signs_detected (list): List of successful Jutsu signs.
        detected (bool): Flag to enable video playing.
    """
    
    def __init__(self, name, success, damage, video_path):
        """
        Initializes the Jutsu class.

        Args:
            name (str): Name of Jutsu technique
            success (list): Jutsu technique pattern.
            damage (int): Amount of damage.
            video_path (Path): Path to the Jutsu animation.
        """
        jutsu_list.append(self)
        self.name = name
        self.success = success
        self.damage = damage
        self.video_path = video_path
        self.hand_signs_detected = []
        self.detected = False
    
    def check_pattern(self, hand_sign, player_id):
        """Validate Jutsu pattern."""
        if len(self.hand_signs_detected) == len(self.success):
            self.detected = True
        elif hand_sign == self.success[len(self.hand_signs_detected)]:
            self.hand_signs_detected.append(hand_sign)
            print(f'{hand_sign.title()} detected for Player {player_id}.')

In [5]:
import copy

class Player:
    """
    A base class for creating Jutsu.
    
    Attributes:
        jutsu_list (list): All Jutsu technique patterns.
        health (int): Player health.
    """
    
    def __init__(self, jutsu_list, player_name):
        """
        Initializes the Player class.

        Args:
            jutsu_list (list): All Jutsu technique patterns.
        """
        self.jutsu_list = copy.deepcopy(jutsu_list)
        self.health = 1000
        self.name = player_name

    def resetJutsu(self):
        """Reset lists of successful Jutsu signs."""
        for jutsu in self.jutsu_list:
            jutsu.hand_signs_detected = []
            jutsu.detected = False
            
    def replenishHP(self):
        self.health = 1000

In [6]:
import os 

name = 'Summoning Jutsu'
success = ['boar', 'dog', 'bird', 'monkey', 'ram']
damage = 200
video_path = os.getcwd() + "\\animation\\summoning_jutsu_vid.mp4"
summoningJutsu = Jutsu(name, success, damage, video_path)
name = 'Fireball Jutsu'
success = ['horse', 'tiger', 'snake', 'ram', 'monkey', 'boar', 'horse', 'tiger']
damage = 500
video_path = os.getcwd() + "\\animation\\fireball_jutsu_vid.mp4"
fireballJutsu = Jutsu(name, success, damage, video_path)
name = 'Chidori Jutsu'
success = ['ox', 'hare', 'monkey']
damage = 100
video_path = os.getcwd() + "\\animation\\chidori_jutsu_vid.mp4"
chidoriJutsu = Jutsu(name, success, damage, video_path)

In [7]:
player1 = Player(jutsu_list, "Player 1")
player2 = Player(jutsu_list, "Player 2")
training_player = Player(jutsu_list, "Dummy")
player_list = [player1, player2, training_player]

def restore_hp(player_list):
    for player in player_list:
        player.replenishHP()
        
restore_hp(player_list)   

## Da Game !!!

In [8]:
import tkinter as tk
import os
from ultralytics import YOLO
import cv2
from PIL import Image, ImageTk
import threading

model = YOLO('best.pt')

class MainMenu:
    def __init__(self, root):        
        self.root = root
        self.root.title("The Next Hokage")
        self.root.geometry("700x500")
        
        # Load the image
        path = os.getcwd() + "\\animation\\bg.png"
        self.background_image = tk.PhotoImage(file=path)

        # Create a label to hold the image
        self.background_label = tk.Label(self.root, image=self.background_image)
        self.background_label.place(x=0, y=0, relwidth=1, relheight=1)
        
        self.create_widgets()

    def create_widgets(self):
        # Adding title
        title_label = tk.Label(self.root, text="The Next Hokage", font=("Comic Sans MS", 40, "bold"), bg="white", fg="#FF8C00")
        title_label.pack(pady=20)  # Padding above the title label
            
        frame = tk.Frame(self.root)
        frame.pack(expand=True)  # Expanding the frame to fill the window
        
        # Creating blank space (padding) above the buttons
        tk.Label(frame, text="Selections", font=("Comic Sans MS", 25, "bold"), bg="white", fg="black").pack()  # Blank label as padding
        
        self.create_button(frame, "Training", self.training_selected, width=15, font=("Comic Sans MS", 12, 'bold'), fg="white", bg="#0080FE")
        self.create_button(frame, "PvP", self.pvp_selected, width=15, font=("Comic Sans MS", 12, 'bold'), fg="white", bg="#0080FE")
        self.create_button(frame, "Exit", self.exit_selected, width=15, font=("Comic Sans MS", 12, 'bold'), fg="white", bg="#0080FE")

    def create_button(self, frame, text, command, **kwargs):
        button = tk.Button(frame, text=text, command=command, **kwargs)
        button.pack(side='top', pady=5)        
            
    def training_selected(self):  ## display (window of) webcam
        print("'Training Mode'")
        
        self.root.withdraw()  # Hide the main window temporarily
        webcam_window1 = tk.Toplevel()  # Create a new window for the webcam
        webcam_window1.title("Training Mode")
        
        label = tk.Label(webcam_window1)
        label.pack()
        
        cap = cv2.VideoCapture(0)  # Open default webcam
        self.show_training(cap, label, webcam_window1)
        self.hp_indictation(webcam_window1, player1)
        if player1.health == 0:
            cap.release()
            self.show_text(label, "Defeat")
            restore_hp(player_list) 
        
        # Close webcam properly
        def close_webcam_window1():
            webcam_window1.destroy()
            cap.release()
            self.root.deiconify()  # Restore the main window when webcam window is closed
            
        webcam_window1.protocol("WM_DELETE_WINDOW", close_webcam_window1)       
        
    def show_training(self, cap, label, window): ## show (playing) webcam

        # threshold
        thres = 0.4
        ret, img = cap.read()
        animate = 0
        if ret:            
            training_frame = cv2.flip(img, 1) # vertically flip image
            results_training = model(training_frame, stream=True, verbose=False) # conf=thres
            training_frame, hand_sign_training = check_hand_sign(results_training)
            defeated, animate = deploy_damage(hand_sign_training, player_list, 3)

            #-----Tkinter GUI------#
            frame = cv2.cvtColor(training_frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            img_tk = ImageTk.PhotoImage(image=img)
            label.img = img_tk
            label.config(image=img_tk)                      
            
        if (animate != 0):
            cap.release()  # Pause webcam
            playVideo(animate)  # Play video
            window.destroy()
            self.training_selected()  # Resume webcam
            
        else:
            window.after(30, lambda: self.show_training(cap, label, window))

   
    def hp_indictation(self, window, player):
        player_label = tk.Label(window, text=f"{player.name}", font=("Comic Sans MS", 20, "bold"), fg="black")
        player_label.place(relx=1, y=0, anchor='ne')  # Positioned at the top-right corner

        hp_label = tk.Label(window, text=f"HP: {player.health}", font=("Comic Sans MS", 20, "bold"), fg="red")
        hp_label.place(relx=1, y=50, anchor='ne')  # Positioned below player_label
    
    def show_text(self, label, text):
        defeated_label = tk.Label(label.master, text=f"{text}", font=("Comic Sans MS", 40), fg="red")
        defeated_label.place(relx=0.5, rely=0.5, anchor='center')  # Position "Defeated" text in the center of the window

    def pvp_selected(self):   
        print("'PvP mode'")
        
        self.root.withdraw()  # Hide the main window temporarily
        
        # Create a new window for displaying player frames
        pvp_window = tk.Toplevel()
        pvp_window.title("PvP Mode")

        # Create labels to display player frames
        player1_label = tk.Label(pvp_window)
        player1_label.pack(side='left')

        player2_label = tk.Label(pvp_window)
        player2_label.pack(side='right')
        
        # start webcam
        cap = cv2.VideoCapture(0)
        self.show_pvp(cap, player1_label, player2_label, pvp_window)
        
        if player1.health == 0:
            cap.release()
            self.show_text(player1_label, "P1 Lost , P2 Won")
            restore_hp(player_list) 
        elif player2.health == 0:
            cap.release()
            self.show_text(player1_label, "P1 Won , P2 Lost")
            restore_hp(player_list) 
            
        # Close webcam properly
        def close_pvp_window():
            pvp_window.destroy()
            cap.release()
            self.root.deiconify()  # Restore the main window when webcam window is closed
            
        pvp_window.protocol("WM_DELETE_WINDOW", close_pvp_window)

            
    def show_pvp(self, cap, player1_label, player2_label, window):
        # threshold
        thres = 0.4
        animate1 = 0; animate2 = 0

        ret, img = cap.read()
    
        if ret:

            # vertically flip image
            img = cv2.flip(img, 1)
            left_frame = img[:, :img.shape[1]//2]
            right_frame = img[:, img.shape[1]//2:]

            results_player1 = model(left_frame, stream=True, verbose=False) # conf=thres
            results_player2 = model(right_frame, stream=True, verbose=False)
            left_frame, hand_sign_player1 = check_hand_sign(results_player1)
            right_frame, hand_sign_player2 = check_hand_sign(results_player2)

            # Convert frames to RGB for displaying in Tkinter
            left_frame_rgb = cv2.cvtColor(left_frame, cv2.COLOR_BGR2RGB)
            right_frame_rgb = cv2.cvtColor(right_frame, cv2.COLOR_BGR2RGB)

            # Convert frames to ImageTk format
            left_img = Image.fromarray(left_frame_rgb)
            left_img_tk = ImageTk.PhotoImage(image=left_img)
            right_img = Image.fromarray(right_frame_rgb)
            right_img_tk = ImageTk.PhotoImage(image=right_img)

            # Update labels with the frames
            player1_label.img = left_img_tk
            player1_label.config(image=left_img_tk)
            player2_label.img = right_img_tk
            player2_label.config(image=right_img_tk)
            self.hp_indictation(player1_label, player1)
            self.hp_indictation(player2_label, player2)

            defeated1, animate1 = deploy_damage(hand_sign_player1, player_list, 1)
            defeated2, animate2 = deploy_damage(hand_sign_player2, player_list, 2)

        if (animate1 != 0):
            cap.release()  # Pause webcam
            playVideo(animate1)  # Play video
            window.destroy()
            self.pvp_selected()  # Resume webcam
            
        elif (animate2 != 0):
            cap.release()  # Pause webcam
            playVideo(animate2)  # Play video
            window.destroy()
            self.pvp_selected()  # Resume webcam

        else:
            window.after(30, lambda: self.show_pvp(cap, player1_label, player2_label, window))


    # exit function
    def exit_selected(self):
        print("'Exit' selected")
        self.root.destroy()  # Close the Tkinter window
        cv2.destroyAllWindows()

def main():
    root = tk.Tk()
    app = MainMenu(root)
    root.mainloop()

if __name__ == "__main__":
    main()


'PvP mode'
Ox detected for Player 2.
Horse detected for Player 2.
Hare detected for Player 2.
Monkey detected for Player 2.
Player 2 deployed Chidori Jutsu!
Player 1 is defeated!
'PvP mode'
'PvP mode'
Ox detected for Player 2.
Ox detected for Player 1.
Horse detected for Player 1.
Hare detected for Player 1.
Monkey detected for Player 1.
Player 1 deployed Chidori Jutsu!
Player 2 is defeated!
'PvP mode'
'Exit' selected
